In [1]:
import gta_prices
import numpy as np
import pandas as pd

In [2]:
def donchian(prices, periods):
    '''Calculate upper, lower, & middle Donchian lines.'''
    for i, period in enumerate(periods.values()):
        df = pd.DataFrame(prices.copy())
        i += 1
        df['upr'] = prices.high.rolling(period).max().shift(periods=1)
        df['lwr'] = prices.low.rolling(period).min().shift(periods=1)
        df['mid'] = 0.5 * (df.upr + df.lwr)
        locals()[f'dc{i}'] = df
    return df


def state(entry_signal, exit_signal, period):
    '''Calculate trade state signals.'''
    df = pd.concat([entry_signal, exit_signal], axis=1)
    df.columns = ['entry', 'exit']
    df['state'] = 0
    for i in range(period, len(df)):
        if df.loc[df.index[i], 'entry'] == 1 \
                and df.loc[df.index[i - 1], 'state'] == 0:
            df.loc[df.index[i], 'state'] = 1
        elif df.loc[df.index[i], 'exit'] == 1:
            df.loc[df.index[i], 'state'] = 0
        else:
            df.loc[df.index[i], 'state'] = df.loc[df.index[i - 1], 'state']
    return df.state

In [3]:
# Trade parameters.
exchange = 'LSE'
tidm = 'HSV'
periods = {'p1': 48, 'p2': 24, 'p3': 12, 'p4': 6}
position_size = 7500
risk_pct = 0.2
commission = 11.95
sduty = 0.5

In [4]:
# Import weekly closing prices.
prices = gta_prices.weekly(exchange, tidm)

In [5]:
# Donchian lines.
# for i, period in enumerate(periods.values()):
#     i += 1
#     globals()[f'p{i}'] = period
#     globals()[f'dc{i}'] = donchian(prices, period)

In [6]:
# dc = pd.concat([dc1, dc2, dc3, dc4], keys=['dc1', 'dc2', 'dc3', 'dc4'])
# dc.loc['dc4']

In [7]:
# Donchian lines, entries & exits for each look back period.
# entry = None
# for i, period in enumerate(periods.values()):
#     i += 1
#     globals()[f'p{i}'] = period
#     df = pd.DataFrame(prices.copy())
#     df['upr'] = prices.high.rolling(period).max().shift(periods=1)
#     df['lwr'] = prices.low.rolling(period).min().shift(periods=1)
#     df['mid'] = 0.5 * (df.upr + df.lwr)
#     df['sys'] = i
#     df['period'] = period
#     if i == 1:
#         df['buy'] = np.where(df.close > df.upr, 1, 0)
#     else:
#         df['buy'] = entry
#     df['sell'] = np.where(df.close < df.mid, 1, 0)
#     df['state'] = state(df.buy, df.sell, period)
#     df['entry'] = np.where(np.logical_and(df.state == 1, df.state.shift(periods=1) == 0), 1, 0)
#     if i == 1:
#         entry = df.entry
#     df['exit'] = np.where(np.logical_and(df.state == 0, df.state.shift(periods=1) == 1), 1, 0)
#     globals()[f'dc{i}'] = df

In [11]:
dc = [f'dc{i + 1}' for i in range(len(periods))]
dc

['dc1', 'dc2', 'dc3', 'dc4']

In [ ]:
# frames = [ process_your_file(f) for f in files ]
# result = pd.concat(frames)